## Incubadora Santander

In [ ]:

import pandas as pd
import numpy as np
import datetime
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from warnings import filterwarnings
from IPython.display import HTML
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

pd.options.display.max_columns = 500

In [ ]:
import statsmodels
import pandas
import numpy
import sklearn

In [ ]:
df.to_csv("reproductoras_pesadas.csv")

In [ ]:
df.head(1)

In [ ]:
# filtrado de datos de acuerdo a sexos
df=df[df.sexoAnimales != "Mixtos"]

In [ ]:
df.fecha = pd.to_datetime(df.fecha)
#df = df[[x.year != 2017 for x in df.fecha]]

df['granjaGalpon'] = ['{} / {}'.format(x, y) for (x, y) in zip(df.granja, df.noDeGalpon)]
#df.idLote = [x if x[-1] != 'A' else x[:-1] for x in df.idLote]
df['pesoAveG'] = [x if x != 0.0 else np.nan for x in df.pesoAveG]

In [ ]:
targets = ['consumoAcumG', 'pesoAveG', 'mortalidadAcumuladaPor',"uniformidadPor","consumoAguaAveDiaMl"]
dl = df[(df.etapa == 'Levante')]

dl_H = dl[dl.sexoAnimales == "Hembras"]
dl_M = dl[dl.sexoAnimales == "Machos"]

In [ ]:
dl_Pivot = pd.merge(dl_H[['consumoAcumG', 'pesoAveG', 'mortalidadAcumuladaPor',"uniformidadPor","consumoAguaAveDiaMl","idLote","edadEnSemanas","granjaGalpon","fecha"]],
                    dl_M[['consumoAcumG', 'pesoAveG', 'mortalidadAcumuladaPor',"uniformidadPor","consumoAguaAveDiaMl","idLote","edadEnSemanas","granjaGalpon"]],
                    left_on=["idLote","edadEnSemanas","granjaGalpon"], right_on=["idLote","edadEnSemanas","granjaGalpon"],
        suffixes=('_Hembras', '_Machos'))

In [ ]:
dl_Pivot["RelaciónPesoMachoHembras"] = [x/y for x,y in zip(dl_Pivot.pesoAveG_Machos,dl_Pivot.pesoAveG_Hembras)]

In [ ]:
plotly_colors = ['rgb(31, 119, 180)', 'rgb(255, 127, 14)',
                         'rgb(44, 160, 44)', 'rgb(214, 39, 40)',
                         'rgb(148, 103, 189)', 'rgb(140, 86, 75)',
                         'rgb(227, 119, 194)', 'rgb(127, 127, 127)',
                         'rgb(188, 189, 34)', 'rgb(23, 190, 207)']

def plot_targetCat_boxplot(dfc, target,Categorical):
    """
    Plot boxplot of input target by cluster
    :param dfc: (pandas.DataFrame) matrix with targets, pcas and cluster labels
    :param target: (str) name of the target to plot
    :param Categorical: (str) name of the categorical variable
    """

    # Asign Colors by Cluster
    levels = dfc[Categorical].unique()
    #colors = dict(zip(clusters, plotly_colors))

    # Set Figure
    fig = go.Figure(

        # Figure Traces
        data = [go.Box(
            y=dfc[dfc[Categorical] == level][target],
            name=level,jitter = 0.3,  pointpos = -1.8,
    boxpoints = 'all', #marker=dict(color=colors[cluster]),
        ) for level in levels],

        # Figure Layout
        layout = go.Layout(
            xaxis=dict(title=target, tickformat='.2f'),
            annotations=[dict(
                text='Distribution of {} by Category in {}'.format(target,Categorical),
                font=dict(size=24),
                showarrow=False, x=0.0, xref='paper', xanchor='left', y=1.15, yref='paper', yanchor='top'
            )]
        )
    )

    # Display Figure
    iplot(fig)

In [ ]:
targets = ['consumoAcumG', 'pesoAveG', 'mortalidadAcumuladaPor',"uniformidadPor","consumoAguaAveDiaMl"]
targets_hembras = ["nacimientoPor","pesoPromHuevoG","posturaPor","produccionHuevosParaIncubarPor","haa","pollosAA","huevoIncAA","conversionHuevoInc"]
for target in targets:
    plot_targetCat_boxplot(df[(df.edadEnSemanas == 60) & (pd.notna(df[target]))],target,"sexoAnimales")
print("Diferencia en resultados para hembras por línea genética")    
for target in targets_hembras:
    plot_targetCat_boxplot(df[(df.edadEnSemanas == 60) & (pd.notna(df[target]))],target,"lineaGenetica")    

In [ ]:
for target in targets:
    fig = go.Figure()
    dataset =dl_Pivot[dl_Pivot.edadEnSemanas == 20]
    fig.add_trace(go.Scatter(x=dataset.fecha, y=dataset[target+"_Hembras"],
                                mode='markers',
                                name=target+"_Hembras",
                                line = {'color':'rgb(0, 0, 122)' }
                                    ))
    fig.add_trace(go.Scatter(x=dataset.fecha, y=dataset[target+"_Machos"],
                                mode='markers',
                                name=target+"_Machos",
                                line = {'color':'rgb(122, 0, 0)' }
                                    ))
    fig['layout'].update(title='Comparación de {} a semana 20 de levante'.format(target))
    fig['layout']['xaxis'].update(title="Fecha")
    fig['layout']['yaxis'].update(title="{}".format(target))
    iplot(fig)

In [ ]:
def histplot(var):
    x = dff[var]
    hist_data = [x]
    group_labels = [var]
    if var== "conversionAlimenticia":
        fig = ff.create_distplot(hist_data, group_labels,bin_size=[.05])
    elif var== "kgm2DiaActivo":
        fig = ff.create_distplot(hist_data, group_labels,bin_size=[.025])    
    else:
        fig = ff.create_distplot(hist_data, group_labels)
    fig['layout'].update(title='Distribución de la variable {}'.format(var))
    iplot(fig, filename='Distplot - {}'.format(var))
for i in range(len(targets)):    
    histplot(targets[i])
histplot("densidadKgm2")    

In [ ]:
def histplotgen(var):
    x1 = df[df.razaGen=="ROSS"][var]
    x2 = df[df.razaGen=="COBB"][var]
    hist_data = [x1,x2]
    group_labels = ['ROSS', 'COBB']

    if var== "conversionAlimenticia":
        fig = ff.create_distplot(hist_data, group_labels,bin_size=[.05,.05])
    elif var== "kgm2DiaActivo":
        fig = ff.create_distplot(hist_data, group_labels,bin_size=[.025,0.025])      
    else:
        fig = ff.create_distplot(hist_data, group_labels)
    fig['layout'].update(title='Distribución de la variable {} según raza'.format(var))
    iplot(fig, filename='Distplot - {}'.format(var))
for i in range(len(targets)):    
    histplotgen(targets[i])
histplotgen("densidadKgm2")     

In [ ]:
def plot_correlation_matrix(dff, title='Correlation Matrix'):
    """
    Plot Correlation Matrix
    :param dff: (pandas.DataFrame) matrix with features to plot
    """

    # Correlation Matrix
    df = dff.dropna(how='any')
    corr_mat = dff.corr()

    # Hide Right Upper Part
    for i in range(len(corr_mat)):
        for j in range(len(corr_mat)):
            if i > j:
                corr_mat.iloc[i, j] = np.nan
    corr_mat = corr_mat[corr_mat.columns.tolist()[::-1]]

    # Set Figure
    fig =  go.Figure(

        # Figure Traces
        data=[go.Heatmap(
            x=corr_mat.columns, y=corr_mat.index, z=corr_mat.values,
            colorscale='Picnic', zmax=1, zmin=-1
        )],

        # Figure Layout
        layout=go.Layout(
            height=400,
            annotations=(
                # Correlation Values as Annotations
                [dict(
                    x=x, y=y,
                    text='' if np.isnan(corr_mat.loc[y][x]) else round(corr_mat.loc[y][x], 2),
                    showarrow=False,
                    font=dict(size=16)
                ) for x in corr_mat.columns for y in corr_mat.index] +

                # Figure Title
                [dict(
                    text=title,
                    font=dict(size=24),
                    showarrow=False, x=0.0, xref='paper', xanchor='left', y=1.15, yref='paper', yanchor='top'
                )]
            ),
            margin=dict(l=150, t=50),
            xaxis=dict(tickangle=-30),
            # autosize=False
        )
    )

    # Display Figure
    iplot(fig)
    

def deal_with_nans(x, thresh=0.7):
    """
    Deal with nan data
    First drop columns for which there isn't enoug data.
    Then fill missing values using median for continuous features and mode for categorical
    Finally drop columns with no variability
    :param x: (pandas.DataFrame) features dataframe
    :param thresh: (float) features with missing values below the thresh are discarted
    :return: (pandas.DataFrame) cleaned dataframe
    """

    # Drop rows with no data
    dff = x.dropna(how='all')

    # Drops columns with not enough data
    dff.dropna(thresh=thresh * len(dff), axis=1, inplace=True)

    # Identify continuous features
    float_vars = [y for y in dff.columns if 'float' in str(dff[y].dtype)]

    # Fill missing values
    for feat in dff.columns:
        if feat in float_vars:
            # Fill continuous features with median
            dff[feat] = dff[feat].fillna(dff[feat].median())
        else:
            # Fill categorical features with mode
            dff[feat] = dff[feat].fillna(dff[feat].mode()[0])

    # Drops columns with no variability
    dff = dff[[y for y in dff.columns if len(dff[y].unique()) > 1]]

    # Returns dataframe with no NaNs
    return dff


def clean_outliers(x, alpha=0.05):
    """
    Put all outliers on the confidence interval limit.
    :param x: (pandas.DataFrame) matrix of features
    :param alpha: (float) significance, 5% by default
    :return: (pandas.DataFrame) matrix with no outliers
    """

    # Cleaning Initialization
    x_c = x.copy()

    # Identification of categorical variables
    cont_vars = [x for x in x_c.columns if 'float' in str(x_c[x].dtype)]

    # If no continuous variables
    if len(cont_vars) == 0:
        return x_c

    # Cleaning
    for var in cont_vars:
        ll = x_c[var].quantile(alpha)
        ul = x_c[var].quantile(1-alpha)
        x_c[var] = [ll if x < ll else (ul if x > ul else x) for x in x_c[var]]

    # Return
    return x_c


def standarize(x):
    """
    Normalize data
    :param x: (pandas.DataFrame) matrix to normalize
    :return: (pandas.DataFrame) normalized matrix
    """

    # Outliers Treatment
    x_oc = clean_outliers(x)

    scale = StandardScaler()
    return pd.DataFrame(scale.fit_transform(x_oc), columns=x.columns, index=x_oc.index)


def get_principal_componets(x, n_pca=None):
    """
    Obtain the principal components from a set of continuous features.
    If the number of components is not especified, the model find the minimum number
    required to explain 90% of variance.
    :param x: (pandas.DataFrame) matrix from which the components are extracted
    :param n_pca: number of components
    :return: (pandas.DataFrame) matrix of components
    """

    # Standarization
    x_std = standarize(x)

    # Identify minimum number of components to explain 90% of the variability
    if n_pca is None:
        pca = PCA(random_state=1234)
        pca.fit(x_std)
        loadings = [round(y, 2) for y in pca.explained_variance_ratio_]
        cumsumload = list(np.cumsum(loadings))
        ncomps = [(y + 1, cumsumload[y]) for y in range(len(cumsumload))]
        n_pca = [y for y in ncomps if y[1] >= 0.9][0][0]

    # Calculate and return pcs
    pca = PCA(n_components=n_pca, random_state=1234)
    dp = pd.DataFrame(
        pca.fit_transform(x_std),
        columns=['PCA '+str(i+1) for i in range(n_pca)],
        index=x.index
    )
    return dp


def clusterize(x, n_clusters=3, pca=False):
    """
    Performs KMeans clusterization.
    If the number of components is not especified, the model finds
    the number of clusters that maximize the silhouette score
    :param x: (pandas.DataFrame) matrix of continuos features used for clusterization
    :param n_clusters: (int) number of clusters
    :param pca: (boolean) True if PCA must be applied before clusterization
    :return: (pandas.DataFrame) matrix of clusters
    """
    # Cleaning
    x_c = deal_with_nans(x)

    n_features = len(x.columns)
    if n_features == 1:
        # Binning
        x_std = np.array(x_c).reshape(-1, 1)
    else:
        # PCA
        x_std = get_principal_componets(x_c) if pca else standarize(x_c)

    # Identify the number of clusters that maximize the silhouette score
    if n_clusters is None:
        cluster_span = [y for y in range(2, 7)]
        ss = []
        for i in cluster_span:
            model = KMeans(n_clusters=i, random_state=1234)
            model = model.fit(x_std)
            mm = pd.DataFrame(model.fit_predict(x_std), columns=['labels'])
            ss.append(round(silhouette_score(x_std, mm['labels']), 3))

        ind = ss.index(max(ss))
        n_clusters = cluster_span[ind]

    # Clusterization
    model = KMeans(n_clusters=n_clusters, random_state=1234)
    model = model.fit(x_std)
    dc = pd.DataFrame(model.fit_predict(x_std), columns=['cluster'], index=x_c.index)

    # If there is a target, the clusters are sorted in accordance to that target
    if 'target' in x_c.columns:
        dc = pd.concat([x_c, dc], axis=1)
        # dc['target'] = x_c['target']
        ds = dc.groupby('cluster').mean()
        ds.sort_values('target', ascending=False, inplace=True)
        cluster_dict = dict(zip(ds.index, ['C' + str(i + 1) for i in range(n_clusters)]))
        dc.cluster = [cluster_dict[y] for y in dc.cluster]
        dc.sort_values('cluster', ascending=True, inplace=True)

    else:
        dc.cluster = dc.cluster + 1
        dc.cluster = ['C'+str(c) for c in dc.cluster]

    # Return
    return dc

### Correlaciones en Producción

In [ ]:
targets_hembras = ["nacimientoPor","pesoPromHuevoG","posturaPor","produccionHuevosParaIncubarPor","haa","pollosAA","huevoIncAA","conversionHuevoInc"]

In [ ]:
corr_targets = {'mortalidadAcumuladaPor_Hembras': 'MORT_H', 'mortalidadAcumuladaPor_Machos': 'MORT_M', 'haa': 'HAA',"nacimientoPor":"NACI_Por","pollosAA":"POLLOSAA","huevoIncAA":"HUEVOINCAA", 'consumoAcumG_Hembras': 'CONS_H', 'consumoAcumG_Machos':'CONS_M', 'pesoAveG_Hembras': 'PESOAVG_H','pesoAveG_Machos': 'PESOAVG_M',"RelaciónPesoMachoHembras":"PESO_M/PESO_H"}
cut_weeks = [25, 60]

In [ ]:
#dp = df[df.etapa == 'Producción']
dp = dp_Pivot

In [ ]:
dc = pd.DataFrame(index=dp.idLote.unique())
for target in corr_targets:
    dv = dp[dp.edadEnSemanas.isin(cut_weeks)]
    dv = dv.pivot_table(index='idLote', columns='edadEnSemanas', values=target)
    dv.columns = [corr_targets[target] + '_S' + str(int(x)) for x in dv.columns]
    
    dc = dc.join(dv)

In [ ]:
dc.drop(columns=['HUEVOINCAA_S25', 'NACI_Por_S25'],inplace=True)

In [ ]:
plot_correlation_matrix(dc, 'Matriz de Correlación de Producción')

## matriz de correlación Hembras

In [ ]:
col_Machos = [x  for x in dc.columns.unique() if "_M" in x]

In [ ]:
plot_correlation_matrix(dc[col_Machos], 'Matriz de Correlación de Producción - Machos')

## matriz de correlación Hembras

In [ ]:
col_Hembras = [x  for x in dc.columns.unique() if x not in col_Machos]
col_Hembras.append("PESO_M/PESO_H_S60")
col_Hembras.append("PESO_M/PESO_H_S25")

In [ ]:
plot_correlation_matrix(dc[col_Hembras], 'Matriz de Correlación de Producción - Hembras')

In [ ]:
def calculate_r_squared(y_real, model):
    y_mean = np.mean(y_real)
    ss_tot = sum([(y - y_mean)**2 for y in y_real])
    ss_res = (model.resid_response**2).sum()
    return round(1 - ss_res/ss_tot, 3)

families = [
    sm.families.Gaussian(),
    sm.families.Binomial(),
    sm.families.Gamma(),
    sm.families.Gaussian(),
    sm.families.InverseGaussian(),
    sm.families.NegativeBinomial(),
    sm.families.Poisson(),
    sm.families.Tweedie(),
]


def perform_regression(x, y, alpha=0.05):
    
    dcc = dc[[x, y]].dropna(how='any')
    dcc = clean_outliers(dcc)

    min_aic = 1e100
    for family in families:
        try:
            model = sm.GLM(dcc[y], sm.add_constant(dcc[x]), family=family)
            model = model.fit()
            model_aic = model.aic
            if model_aic < min_aic:
                min_aic = model_aic
                best_model = model
                best_family = family
        except:
            continue

    print(best_model.summary())
    r2 = calculate_r_squared(dcc[y], best_model)
    print('R2: ', r2)
    
    significance = best_model.pvalues[x] < alpha and r2 > 0.2
    preds = pd.DataFrame()
    preds[x] = dcc[x]
    preds[y] = best_model.predict()
    
    return significance, preds

def perform_ols_regression(dataset, x, y):
    dc = dataset.copy()
    dcc = dc[[x, y]].dropna(how='any')
    dcc = clean_outliers(dcc)
    
    mod = sm.OLS(dcc[y],  sm.add_constant(dcc[x]))
    mod = mod.fit()
    print(mod.summary())
    
    normal = sm.stats.jarque_bera(mod.resid)[1] > 0.01
    mod_sig = mod.f_pvalue < 0.05
    coef_sig = mod.pvalues[x] < 0.05
    r2 = mod.rsquared
    
    preds = pd.DataFrame()
    preds[x] = dcc[x]
    preds[y] = mod.predict()
    
    return normal and mod_sig and coef_sig, preds, round(r2,2)
    

def plot_scatter(dataset, x, y, reg=True):
    dc = dataset.copy()
    dcc = dc[[x, y]].dropna(how='any')
    dcc = clean_outliers(dcc)
    data = []
    
    # Scatter
    data.append(go.Scatter(
        x=dcc[x], y=dcc[y], name='points', mode='markers'
    ))
    
    # Regression
    if reg:
        sig, preds, r2 = perform_ols_regression(dataset, x, y)
        if sig:
            preds.sort_values(x, inplace=True)
            data.append(go.Scatter(
                x=preds[x], y=preds[y], name='reg + (R2: {})'.format(r2), mode='lines', line=dict(dash='dash')
            ))
    
    # layout
    lay = go.Layout(
        xaxis=dict(title=x),
        yaxis=dict(title=y)
    )
    
    iplot(dict(layout=lay, data=data))
        

In [ ]:
plot_scatter(dataset=dc, x='MORT_H_S60', y='HAA_S60', reg=False)
plot_scatter(dataset=dc, x='PESO_M/PESO_H_S60', y='HAA_S60', reg=False)

In [ ]:
plot_scatter(dataset=dc, x='MORT_H_S60', y='POLLOSAA_S60', reg=False)
plot_scatter(dataset=dc, x='MORT_M_S60', y='POLLOSAA_S60', reg=False)
plot_scatter(dataset=dc, x='PESO_M/PESO_H_S60', y='POLLOSAA_S60', reg=False)

In [ ]:
plot_scatter(dataset=dc, x='MORT_H_S60', y='HUEVOINCAA_S60', reg=False)
plot_scatter(dataset=dc, x='MORT_M_S60', y='HUEVOINCAA_S60', reg=False)
plot_scatter(dataset=dc, x='PESO_M/PESO_H_S60', y='HUEVOINCAA_S60', reg=False)


In [ ]:
def plot_clusters(dw, cluster_targets, title='Coordinates'):
    dataset = dw.copy()

    dataset = dataset[cluster_targets]
    dataset = deal_with_nans(dataset)
    dataset['cluster'] = clusterize(dataset, pca=True)

    # Soer Clusters by label
    sort_label = cluster_targets[0]
    dsort = dataset.groupby('cluster').median()
    dsort.sort_values(sort_label, ascending=False, inplace=True)
    cluster_translator = dict(zip(dsort.index, ['C3', 'C2', 'C1']))

    dataset['cluster'] = [cluster_translator[x] for x in dataset.cluster]
    dataset.sort_values('cluster', ascending=True, inplace=True)

    dfc = dataset.copy()

    # Identify targets
    targets = [x for x in dfc.columns if 'PCA ' not in x and x != 'cluster']
    x_t = dfc[targets]
    y = dfc['cluster']

    # Get Ordered Targets
    ordered_targets = cluster_targets

    # Asign Colors by Cluster
    clusters = dfc.cluster.unique().tolist()
    colors = {
        'C3': '#2ca02c',
        'C2': '#ff7f0e',
        'C1': '#d62728'
    }

    # Set Figure
    fig = go.Figure(

        # Figure Traces
        data=[go.Parcoords(
            line=dict(
                color=[clusters.index(x) for x in dfc['cluster']],
                colorscale=[[clusters.index(x)/2, colors[x]] for x in clusters],
            ),
            dimensions= [dict(
                label='Cluster',
                values=[int(x.replace('C', '')) for x in dfc['cluster']],
                tickvals=[int(x.replace('C', '')) for x in clusters],
                ticktext=clusters
            )] + [dict(
                label=y,
                values=dfc[y],
                tickformat='.2f'
            ) for y in ordered_targets]
        )],

        # Figure Layout
        layout=go.Layout(
            annotations=[dict(
                text=title,
                font=dict(size=24),
                showarrow=False, x=0.0, xref='paper', xanchor='left', y=1.30, yref='paper', yanchor='top'
            )],
            margin=dict(b=50)
        )
    )

    # Display Figure
    iplot(fig)

In [ ]:
dc.columns.unique()

In [ ]:
targets = ['PESOAVG_M_S25', #'PESOAVG_H_S25',
           'POLLOSAA_S60', 'HUEVOINCAA_S60', 'HAA_S60', 'MORT_H_S60']
plot_clusters(dc, targets, '')

In [ ]:
granjas = df[['idLote','granja']]
granjas.drop_duplicates(inplace=True)
granjas.set_index('idLote',inplace=True)
granjas.head()

In [ ]:
dataset = dc.copy()

dataset = dataset[['PESOAVG_M_S25', 'PESOAVG_H_S25', 'POLLOSAA_S60', 'HUEVOINCAA_S60', 'HAA_S60', 'MORT_H_S60']]
dataset = deal_with_nans(dataset)
dataset['cluster'] = clusterize(dataset, pca=True)
dataset['granja'] =[granjas.loc[x,'granja'] for x in dataset.index]
dataset.head()

In [ ]:
cluster_granja = dataset.pivot_table(index='granja',columns='cluster',values='HAA_S60',aggfunc='count', margins=True)
cluster_granja.fillna(0,inplace=True)
for cluster in cluster_granja.columns:
    cluster_granja[cluster]=round((cluster_granja[cluster]/cluster_granja['All'])*100,1)
cluster_granja.drop('All',inplace=True)
cluster_granja.drop('All',inplace=True, axis=1)
cluster_granja

### Correlación entre Levante y Producción

In [ ]:
plot_correlation_matrix(dpl,'Matriz de Correlación de Levante y  Producción')

In [ ]:
col_Machos_dlp = [x  for x in dpl.columns.unique() if "_M" in x]

In [ ]:
plot_correlation_matrix(dpl[col_Machos_dlp],'Matriz de Correlación de Levante y  Producción - Machos')

In [ ]:
col_Hembras_dlp = [x  for x in dpl.columns.unique() if x not in col_Machos_dlp]

In [ ]:
plot_correlation_matrix(dpl[col_Hembras_dlp],'Matriz de Correlación de Levante y  Producción - Hembras')

In [ ]:
dpl.corr()["HAA"].sort_values()

In [ ]:
plot_scatter(dpl, 'PESO_H_20', 'HAA')

In [ ]:
targets = ['PESO_M_20', "MORT_H","MORT_M",'HAA', 'CONV_H']
plot_clusters(dpl, targets, '')

In [ ]:
plot_scatter(dpl, 'PESO_M_20', 'HAA')

In [ ]:
targets = ['PESO_H_20','PESO_M_20','PESO_M_25', 'HAA', 'CONV_H']
plot_clusters(dpl, targets, '')

In [ ]:
#dpl

In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')